In [7]:
import time

class URLShortener:
    def __init__(self, default_ttl =60):
        self.url_to_id ={}
        self.id_to_url ={}
        self.url_expiry = {}
        self.click_count ={} #New dictionary to track hits
        self.counter =1
        self.default_ttl =default_ttl
        self.alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

    def encode(self, long_url, ttl =None, custom_alias = None):
        if long_url in self.url_to_id:
            short_id = self.url_to_id[long_url]
        else:
            # Handle custom alias
            if custom_alias:
                if custom_alias in self.id_to_url:
                    return f"Alia '{custom_alias}' is already taken"
                short_id =custom_alias
            else:
                short_id = self._int_to_base62(self.counter)
                self.counter+=1
                
            short_id =self._int_to_base62(self.counter)
            self.url_to_id[long_url] = short_id
            self.id_to_url[short_id] = long_url
            expiry_time =time.time() + (ttl if ttl else self.default_ttl)
            self.url_expiry[short_id] = expiry_time
            self.counter  +=1
        return f"https://short.ly/{short_id}"

    

    def decode(self, short_url):
        short_id = short_url.split("/")[-1]

        # Check if expired
        if short_id not in self.url_expiry:
            return "URL not found"
        if time.time() > self.url_expiry[short_id]:
            return "URL has expired"

        #track the click
        self.click_count[short_id] = self.click_count.get(short_id, 0)+1
        return self.id_to_url.get(short_id, "URL not found")


    def get_analytics(self, short_url):
        short_id = short_url.split("/")[-1]
        url = self.id_to_url.get(short_id)
        clicks = self.click_count.get(short_id,0)
        expiry = self.url_expiry.get(short_id, 0)
        expires_in = max(0, int(expiry - time.time()))

        return {
            "original_url": url,
            "short_url":short_url,
            "clicks":clicks,
            "expires_in_sec":expires_in}


    def _int_to_base62(self,num):
        base62 =[]
        base = len(self.alphabet)
        while num >0:
            num, rem = divmod(num, base)
            base62.append(self.alphabet[rem])
        return ''.join(reversed(base62))

    def _base62_to_int(self,str_id):
        base = len(self.alphabet)
        num =0
        for char in str_id:
            num = num*base + self.alphabet.index(char)
        return num


### Example Usage

In [10]:
shortener = URLShortener()

url1 = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
url2 = "https://pythontutor.com/render.html#mode=display"

short1 = shortener.encode(url1)
short2 = shortener.encode(url2)

print("Short URLs:", short1, short2)

print("Original:", shortener.decode(short1))
print("Original:", shortener.decode(short2))

Short URLs: https://short.ly/c https://short.ly/e
Original: https://www.youtube.com/watch?v=dQw4w9WgXcQ
Original: https://pythontutor.com/render.html#mode=display


### Create a Short URL With 5 Seconds TTL 

In [13]:
shortener =URLShortener()

url ="https://pythontutor.com/render.html#mode=display"
short_url = shortener.encode(url, ttl=5)  # expires in 5 seconds
print("Short URL:", short_url)

print("Trying to decode...")
print(shortener.decode(short_url))  # should work

time.sleep(6)

print("After expiry...")
print(shortener.decode(short_url))  # should say expired

Short URL: https://short.ly/c
Trying to decode...
https://pythontutor.com/render.html#mode=display
After expiry...
URL has expired


### Test the Custom Alias

In [15]:
shortener =URLShortener()

custom_url = shortener.encode("https://openai.com", custom_alias="openai")
print(custom_url)

https://short.ly/b


### Testing for click counter

In [19]:
shortener = URLShortener()

url = "https://openai.com"
short = shortener.encode(url, custom_alias="openai")

# Simulate visits
shortener.decode(short)
shortener.decode(short)
shortener.decode(short)

# Get analytics
print(shortener.get_analytics(short))

{'original_url': 'https://openai.com', 'short_url': 'https://short.ly/b', 'clicks': 3, 'expires_in_sec': 60}
